In [3]:
import ipytone

## Transport

The Tone.js global timeline is exposed in Python with the
`transport.Transport` singleton class, which may be accessed via
`ipytone.transport`:

In [4]:
t = ipytone.transport

> important
Like the audio context used by ipytone and Tone.js, the
`Transport` timeline
is exposed globally in the front-end. Consequently, if two or more notebooks
with independent kernels are open in the same JupyterLab tab, they
will all act on the same timeline. 


### Playback state, position, bpm, time signature...

The playback state of the timeline is controlled by the
`Transport.start`,
`Transport.pause` and
`Transport.stop` methods.

In [5]:
t.start().stop("+1")

Transport()

The `position` property can be used to move the current "cursor" along the
timeline. It accepts different kinds of values, e.g.,

- a string in the form of `"Bars:Beats:Sixteenths"`
- a string like `"4m"` (four measures from the beginning of the timeline) or "8n" (8-notes)
- a float (number of seconds from the beginning of the timeline)

See the [Tone.js Wiki](https://github.com/Tonejs/Tone.js/wiki/Time) for more details.

In [6]:
t.position

'0:0:0'

It is possible to define a limited segment along the timeline that will be played in loop:

In [15]:
t.loop = True
t.loop_start = "4:0:0"
t.loop_end = "8:0:0"

`Transport` has also properties for setting the BPM or
the time signature:

In [10]:
t.bpm

Param(value=120.0, units='bpm')

In [11]:
t.time_signature

4

## Basic scheduling

Basic event scheduling can be done either using
`Transport` with callbacks or using context managers
provided by ipytone.

Let's first create an instrument

In [12]:
synth = ipytone.Synth(volume=-5).to_destination()

In [13]:
synth.trigger_attack_release("C4", "16n")

Synth()

### Using callbacks

Callbacks must accept one `time` argument (in seconds) and usually implement in
their body all the events that we want be triggered at that time (e.g.,
instrument note, parameter automation, etc.).

In [23]:
def callback(time):
    synth.trigger_attack_release("C4", "16n", time=time)

Then we can pass it to one of the `Transport.schedule`,
`Transport.schedule_repeat` and
`Transport.schedule_once` methods of
`Transport`, e.g.,

In [24]:
# schedule the call repeatedly at every 4th bar note
event_id = t.schedule_repeat(callback, "4n")

In [25]:
t.start().stop("+2m")

Transport()

Those schedule methods return an event id that can be used later to remove it
from the transport timeline with the `Transport.clear`,
method, e.g.,

In [27]:
t.clear(event_id)

Transport()

In [28]:
# no scheduled event, no sound
t.start().stop("+2m")

Transport()

> important:
Ipytone doesn't handle those callbacks like you might expect,
i.e., like functions called at each of the scheduled times. Instead, ipytone
uses some tricks internally to reconstruct (more-or-less) equivalent callbacks
in the front-end before passing it to Tone.js scheduling functions.

As a consequence, Python callbacks have some limitations compared to Tone.js /
JS callbacks. More specifically, the Python code inside a callback will be
executed only once. Here is a bad example that won't behave as we'd like:

```python
import random

def callback(time):
    # This will randomly choose one note once for all repeated events!! 
    note = random.choice(["C4", "G4", "A4"]) 
    synth.trigger_attack_release(note, "16n", time=time)
```

There's also very limited support for making operations with the `time` argument
(currently, only addition is supported).
````

### Using contexts

For convenience, the same scheduling operations can be achieved using context
managers. For example:

In [29]:
with ipytone.schedule_once("4n") as (time, event_id):
    synth.trigger_attack_release("A4", "16n", time=time)
    synth.trigger_attack_release("A5", "16n", time=time + "16n")

In [30]:
t.start().stop("+2m")

Transport()

Note that unlike the example above, the two note triggers here are scheduled
only once so they won't be re-triggered after stopping and restarting the
transport

In [31]:
# no scheduled event, no sound
t.start().stop("+2m")

Transport()

## Advanced scheduling

Ipytone also exposes Tone.js event classes for more advanced and handy
scheduling.

### Loop

`Loop` is a simplified event that is looped by default at a
user-defined interval. Like in basic scheduling, it accepts a callback with one
`time` argument.

In [ ]:
loop = ipytone.Loop(callback=callback, interval="8n")

In [ ]:
loop.start()

In [ ]:
loop.stop()

In [ ]:
loop.cancel()

### Sequence

`Sequence` is an alternative to `Part` where the
note events are evenly spaced at a given subdivision.

In [ ]:
seq = ipytone.Sequence(
    callback=event_clb,
    events=["A4", "C4", "B4", "A5"],
    subdivision="8n",
)

A sequence is looped by default.

In [ ]:
seq.start()

Events may be nested (the interval within a nested array corresponds to the
parent subdivision divided by the length of the nested array).

In [ ]:
seq.events = ["A4", ["C4", "E4", "D4"], "B4", "A5"]

Blank intervals may be defined by `None` array elements:

In [ ]:
seq.events = ["A4", ["C4", "E4", "D4"], "B4", None]

In [ ]:
seq.stop()

### Pattern

`Pattern` is like an arpeggiator, it cycles trough an sequence
of notes with a given pattern.

In [ ]:
pat = ipytone.Pattern(
    callback=event_clb,
    values=["C3", "E3", "G3"],
    pattern="upDown",
)

In [ ]:
pat.start()

In [ ]:
pat.pattern="up"

In [ ]:
pat.stop()

### Dispose events

Like audio nodes, events may also be disposed.

In [ ]:
event.dispose()
loop.dispose()
seq.dispose()
pat.dispose()